In [31]:
import pandas as pd
import numpy as np
import wave
import librosa
import cv2
import os
import glob
import pickle
import joblib
from tqdm import tqdm
import IPython.display
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.io as pio
init_notebook_mode(connected=True)
pio.renderers.default = "notebook_connected"
import warnings
warnings.simplefilter("ignore")

import cosine_similarity as cs
import freq_dataset as fd
cos = cs.CosineSimilarity()
freq = fd.MakeFreqDataset()

s_path = "../../sounds/"

In [18]:
def calc_length(path_ls):
    length_ls = []
    for p in path_ls:
        try:
            audio = wave.open(p)
            frames = audio.getnframes()
            rate = audio.getframerate()
            duration = frames / float(rate)
        except:
            duration = -1
        length_ls.append(duration)
    return length_ls

In [23]:
result_df = pd.read_csv("../files/freq_final_dataset.csv", encoding='utf-8-sig')
result_df.drop(["Unnamed: 0", "label"], axis=1, inplace=True)
result_df['path'] = [p.replace('original/clean', 'original_clean') for p in result_df['path']]
result_df['path'] = [p.replace('original/noise', 'original_noise') for p in result_df['path']]
result_df['path'] = [p.replace('../', '') for p in result_df['path']]
path_ls = ["../../"+p for p in result_df['path']]
result_df['duration'] = calc_length(path_ls)
result_df['path'] = [p.replace('human/read_women', 'human_voice/read_women') for p in result_df['path']]
result_df['path'] = [p.replace('human/read_men', 'human_voice/read_men') for p in result_df['path']]
result_df['path'] = [p.replace('human/영어리딩', 'human_voice/영어리딩') for p in result_df['path']]
result_df['path'] = [p.replace('human/original_noise', 'human_voice/original_noise') for p in result_df['path']]
result_df['path'] = [p.replace('human/original_clean', 'human_voice/original_clean') for p in result_df['path']]
result_df['cate1'] = [p.split('/')[-3] for p in result_df['path']]
result_df['cate2'] = [p.split('/')[-1][:3] if p.split('/')[-2] == 'cats_dogs' else p.split('/')[-2] for p in result_df['path']]
result_df['path'] = path_ls
result_df

,name,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,path,noise,duration,cate1,cate2
0,cat_119,0.284324,0.080283,0.116524,0.035035,1061.271547,1.547536e+05,1329.783998,44388.201164,2001.603846,...,42.138844,5.225847,39.697083,5.260680,33.465557,../../sounds/animal/cats_dogs/cat_119.wav,clean,4.316000,animal,cat
1,dog_barking_100,0.426439,0.109916,0.032564,0.004489,1626.752143,1.490662e+05,1644.495521,70639.091223,3268.740234,...,36.275764,-2.387175,70.726715,3.296551,93.897987,../../sounds/animal/cats_dogs/dog_barking_100.wav,clean,1.854062,animal,dog
2,cat_54,0.203641,0.089293,0.181991,0.005883,2344.144335,3.235486e+05,1503.098172,33353.104390,3627.886573,...,70.673302,17.618805,178.355392,0.816697,176.787277,../../sounds/animal/cats_dogs/cat_54.wav,clean,1.084500,animal,cat
3,cat_15,0.404880,0.133895,0.097528,0.006318,1510.993042,3.345212e+05,1674.112362,258817.119037,3113.328156,...,175.291428,5.300216,83.345917,6.450485,57.837170,../../sounds/animal/cats_dogs/cat_15.wav,clean,2.948000,animal,cat
4,cat_30,0.404268,0.094998,0.072871,0.005941,1547.687106,4.278606e+04,1453.063485,79612.852737,2804.668469,...,52.483185,0.625472,51.724159,3.752055,45.712337,../../sounds/animal/cats_dogs/cat_30.wav,clean,12.000000,animal,cat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22499,UO62_2_3_0_19_0_0_0,0.596687,0.063218,0.017606,0.000238,3012.822673,1.413769e+06,2055.704305,183494.121478,5502.331543,...,61.895054,4.017175,53.293667,-3.685825,27.531643,../../sounds/human/read_women/4_2835.wav,NaN,3.111474,human_voice,read_women
22500,jYYHDnEnnDY.0532,0.480263,0.101976,0.073356,0.005532,1546.017759,5.349460e+05,2091.387153,480797.826176,3599.353096,...,53.615490,-2.828741,37.364723,0.695443,44.442627,../../sounds/human/read_men/3LR5IpiEvSc.1059.wav,NaN,2.280000,human_voice,read_men
22501,QHOC_5_9_0_23_0_0_0,0.562584,0.066113,0.016748,0.000298,1873.343338,5.913499e+05,1825.989854,68530.194293,3769.387207,...,39.740143,1.627433,28.773853,-3.702916,46.514179,../../sounds/human/read_men/nVToV-zn_C4.0677.wav,NaN,3.571000,human_voice,read_men
22502,4_2566,0.341139,0.119525,0.062649,0.002389,1882.043797,1.079272e+06,2124.615936,461103.723759,3737.829425,...,128.742050,-1.848811,201.917435,1.630757,307.923187,../../sounds/human/read_women/3_4288.wav,NaN,3.343673,human_voice,read_women


In [29]:
result_df.groupby('cate1').count().sort_values('name')

,name,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,path,noise,duration,cate2
cate1,,,,,,,,,,,,,,,,,,,,,
nature,216,216,216,216,216,216,216,216,216,216,...,216,216,216,216,216,216,216,216,216,216
human,925,925,925,925,925,925,925,925,925,925,...,925,925,925,925,925,925,925,0,925,925
life,1219,1219,1219,1219,1219,1219,1219,1219,1219,1219,...,1219,1219,1219,1219,1219,1219,1219,0,1219,1219
animal,2110,2110,2110,2110,2110,2110,2110,2110,2110,2110,...,2110,2110,2110,2110,2110,2110,2110,1105,2110,2110
human_voice,18034,18034,18034,18034,18034,18034,18034,18034,18034,18034,...,18034,18034,18034,18034,18034,18034,18034,567,18034,18034


In [30]:
new_df = pd.read_csv("../files/freq_dataset_cats_dogs_noise.csv", encoding='utf-8-sig')
new_df.drop(["Unnamed: 0"], axis=1, inplace=True)
new_df['noise'] = [n[-2:] for n in new_df['name']]
new_df['name'] = [n[:-4] for n in new_df['name']]
new_df['cate1'] = 'animal'
new_df['cate2'] = [n[:3] for n in new_df['name']]
new_df['path'] = [f"../sounds/animal/{new_df['cate2'].iloc[i]}/{new_df['name'].iloc[i]}__{new_df['noise'].iloc[i]}.wav" for i in range(len(new_df))]
new_df

,name,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,noise,cate1,cate2,path
0,cat_117,0.584179,0.075645,0.005973,0.000013,1796.491535,134654.955214,1904.593769,59699.065283,4079.642902,...,3.176553,69.321602,0.475793,49.131870,4.910329,40.988270,r1,animal,cat,../sounds/animal/cat/cat_117__r1.wav
1,cat_117,0.672787,0.040813,0.014639,0.000013,1295.793794,54623.776737,1414.065667,20623.577854,2581.332060,...,0.743420,19.362394,-0.051691,20.939856,3.871450,14.643286,r3,animal,cat,../sounds/animal/cat/cat_117__r3.wav
2,cat_117,0.716626,0.037021,0.029385,0.000137,1022.911679,51851.118421,1499.095165,18285.309789,2679.265427,...,4.209167,17.235205,6.297580,22.045244,2.776757,22.462952,r5,animal,cat,../sounds/animal/cat/cat_117__r5.wav
3,cat_117,0.320546,0.091558,0.113652,0.004641,410.720773,11640.462744,620.958255,43042.205815,651.952564,...,-3.064952,20.002291,-4.061711,23.198385,-2.655361,23.684460,r7,animal,cat,../sounds/animal/cat/cat_117__r7.wav
4,cat_117,0.730912,0.029885,0.037549,0.000132,1166.206190,25181.564131,1459.317513,6109.982161,2893.653414,...,-0.076047,12.481139,11.286463,11.627664,-3.539403,13.164918,r9,animal,cat,../sounds/animal/cat/cat_117__r9.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,cat_167,0.449150,0.104139,0.098204,0.010885,1739.673465,307872.386884,1499.061577,60787.775290,3487.837452,...,-1.462144,97.661026,4.022102,116.294685,1.840472,123.577728,r1,animal,cat,../sounds/animal/cat/cat_167__r1.wav
1381,cat_167,0.467448,0.109826,0.080989,0.006021,1501.144196,443813.737597,1443.648459,64621.875352,3128.541172,...,0.296841,77.154022,6.020858,85.108788,2.409565,89.625381,r3,animal,cat,../sounds/animal/cat/cat_167__r3.wav
1382,cat_167,0.486101,0.110797,0.067886,0.002475,1374.128453,400666.618674,1319.372099,100470.864545,2668.992630,...,-1.725178,44.937675,5.077783,49.057575,0.635597,56.862053,r5,animal,cat,../sounds/animal/cat/cat_167__r5.wav
1383,cat_167,0.487711,0.111753,0.043951,0.000880,1444.546090,366914.613563,1445.836783,71598.580037,3063.233508,...,0.644646,57.623772,6.381130,66.249504,2.445853,71.138428,r7,animal,cat,../sounds/animal/cat/cat_167__r7.wav


In [33]:
dtc_79 = joblib.load('models/DTC_79.pkl')
y_pred = dtc_79.predict(new_df.drop(['name', 'path', 'noise', 'cate1', 'cate2'], axis=1))
y_pred

array(['human_voice', 'nature', 'animal', ..., 'animal', 'animal', 'life'],
      dtype=object)

In [42]:
check_df = new_df[['noise', 'cate1', 'cate2']].copy()
check_df['y_pred'] = y_pred
check_df.groupby(['y_pred', 'noise']).count()

cate1  cate2
y_pred      noise              
animal      r1       235    235
            r3       225    225
            r5       199    199
            r7       170    170
            r9       127    127
human       r1         7      7
            r3         9      9
            r5        20     20
            r7        20     20
            r9        18     18
human_voice r1        18     18
            r3        23     23
            r5        37     37
            r7        52     52
            r9        65     65
life        r1        14     14
            r3        10     10
            r5        15     15
            r7        27     27
            r9        49     49
nature      r1         3      3
            r3        10     10
            r5         6      6
            r7         8      8
            r9        18     18

In [ ]:
cat_df = result_df[result_df['cate2']=='고양이 소리'].copy()
dog_df = result_df[result_df['cate2']=='개 소리'].copy()
dog_df['cate2'] = [c.replace("개 소리", "dog") for c in dog_df['cate2']]
cat_df['cate2'] = [c.replace("고양이 소리", "cat") for c in cat_df['cate2']]
life_df = result_df[result_df['cate1']=='life'].copy().sample(n=500, random_state=135)
nature_df = result_df[result_df['cate1']=='nature'].copy()
voice_n_df = result_df[result_df['cate2']=='original_noise'].copy()
voice_c_df = result_df[result_df['cate2']=='original_clean'].copy()
len(cat_df), len(dog_df), len(nature_df)

In [ ]:
### human_voice: 영어리딩 빼고, 2000개 랜덤 샘플링
voice_random = result_df[result_df['cate1']=='human_voice'].copy()
voice_random.drop(result_df[result_df['cate2']=='영어리딩'].index, axis=0, inplace=True)
voice_random = voice_random.sample(n=500, random_state=135)
voice_random

In [ ]:
# result_df = result_df.drop(result_df[result_df['cate1']=='human_voice'].index, axis=0)
result_df = pd.concat([cat_df, dog_df, life_df, nature_df,voice_n_df, voice_c_df])

In [ ]:
count_df = result_df[['cate1', 'cate2','path']].groupby(['cate1', 'cate2']).count()
count_df = count_df.reset_index()
count_df[count_df['cate1']=='human_voice'].sort_values(by='path', ascending=False)

In [ ]:
tmp_df = result_df.drop(['name', 'path', 'noise', 'cate2'], axis=1)
tmp_df.groupby(['cate1']).count()

In [ ]:
for c in tmp_df.columns.tolist():
    fig = px.box(tmp_df, x='cate1', y=c)
    fig.show()

In [ ]:
tmp_df = result_df.set_index('cate1', drop=True)
tmp_df = tmp_df.drop(['name', 'path', 'noise', 'cate2'], axis=1)
cos_df = cos.make_cos_df(tmp_df)
cos_df

---
# HDBSCAN

In [ ]:
import hdbscan
from sklearn.manifold import TSNE
import seaborn as sns

In [ ]:
tsne = TSNE()
tmp_df = result_df.set_index('path', drop=True)
tmp_df = tmp_df.drop(['name', 'cate1', 'cate2', 'noise'], axis=1)
tsne_fit = tsne.fit_transform(tmp_df)
tsne_df = pd.DataFrame(tsne_fit, index=tmp_df.index, columns=['x', 'y'])

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, min_samples=1, gen_min_span_tree=True,prediction_data=True)
clusterer.fit(tsne_df)
# 예측 데이터 생성
pred_hds = clusterer.fit_predict(tsne_df)
print(np.unique(pred_hds))
clusterer.condensed_tree_.plot(select_clusters=True, selection_palette=sns.color_palette())
hdbscan_df = tsne_df.copy()
hdbscan_df['hdbscan'] = pred_hds
hdbscan_df['cate1'] = result_df['cate1'].tolist()
hdbscan_df['cate2'] = result_df['cate2'].tolist()
hdbscan_df

In [ ]:
hdbscan_df.groupby(['cate1', 'cate2', 'hdbscan']).count()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.scatter(hdbscan_df['x'], hdbscan_df['y'], c=hdbscan_df["hdbscan"].tolist(), s=20, cmap="tab20", alpha=0.6)
ax.set_title("HDBSCAN", fontsize=30)
fig.show()

In [ ]:
px.scatter(hdbscan_df, x='x', y='y',color='cate1', hover_name='hdbscan')

-----

In [ ]:
hdbscan_df

In [ ]:
tmp = hdbscan_df[hdbscan_df['cate1']=='human_voice'].sort_values('x')
tmp['tag'] = [p.split('/')[-2] for p in tmp.index]
tmp['name'] = [p.split('/')[-1][:-4] for p in tmp.index]
px.scatter(tmp, x='x', y='y',color='tag', hover_name='name')

In [ ]:
tmp = hdbscan_df[hdbscan_df['cate1']=='animal'].sort_values('x')
tmp['tag'] = [p.split('/')[-2] for p in tmp.index]
tmp['name'] = [p.split('/')[-1][:-4] for p in tmp.index]
px.scatter(tmp, x='x', y='y',color='tag', hover_name='name')

---

# 분류

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score 
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, plot_confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

import seaborn as sns

In [ ]:
def play_song(path):
    (file_dir, file_id) = os.path.split(path)
    print(file_id[:-4])
    print(IPython.display.display(IPython.display.Audio(path)))
def play_song_ls(cm_lr_df):
    for idx, path in enumerate(cm_lr_df['path']):
        print(cm_lr_df['pred'].iloc[idx])
        play_song(path)

In [ ]:
def model_fit_kfold(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cf = confusion_matrix(y_test, y_pred)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)
    print("accuracy:", round(accuracy_score(y_test, y_pred),6))
    print("recall:", round(recall_score(y_test, y_pred, labels=np.unique(y), average='macro'),6))
    print("f1 score:", round(f1_score(y_test, y_pred, labels=np.unique(y), average='macro'),6))
    print("Cross Validation Score:", cross_val_score(model, X, y, cv=kfold))
    sns.heatmap(cf, annot=True, xticklabels=np.unique(y), yticklabels=np.unique(y), fmt="d", cmap="Blues")
    
    return y_pred

In [ ]:
def check_false_pred(y_pred):
    cm_lr_df = pd.DataFrame()
    cm_lr_df['answer'] = y_test
    cm_lr_df['pred'] = y_pred
    cm_lr_df['result'] = y_test == y_pred
    cm_lr_df['path'] = X_test.index
    cm_lr_df['cate1'] = [result_df[result_df['path']== p]['cate1'].iloc[0] for p in cm_lr_df['path']]
#     cm_lr_df['noise'] = [result_df[result_df['path']== p]['noise'].iloc[0] for p in cm_lr_df['path']]
    cm_lr_df['name'] = [f"{p.split('/')[-2]}__{p.split('/')[-1][:-4]}" for p in cm_lr_df['path']]
#     [l.split('/')[-1]for l in cm_lr_df['path']]
    cm_lr_df = cm_lr_df[cm_lr_df['result']==False].sort_values(['answer', 'cate1','name'])
    return cm_lr_df

In [ ]:
tmp_df = result_df.set_index('path', drop=True)
tmp_df = tmp_df.drop(['name', 'cate1','cate2', 'noise'], axis=1)
X = tmp_df
# y = result_df['label']
y = ['human_voice' if label =='human_voice' else 'Not_human' for label in result_df['cate1']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13, shuffle=True, stratify=y)

In [ ]:
tmp_df = result_df.set_index('path', drop=True)
tmp_df = tmp_df.drop(['name', 'cate1', 'cate2', 'noise'], axis=1)
X = tmp_df
y = result_df['cate1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13, shuffle=True, stratify=y)

In [ ]:
np.unique(y)

In [ ]:
lr_y_pred = model_fit_kfold(LogisticRegression())
cm_lr_df = check_false_pred(lr_y_pred)
cm_lr_df

In [ ]:
dt_y_pred = model_fit_kfold(DecisionTreeClassifier())
cm_dt_df = check_false_pred(dt_y_pred)
cm_dt_df

In [ ]:
xgb_y_pred = model_fit_kfold(XGBClassifier(num_class=5,
                                           learning_rate=0.1,
                                           eval_metric='mlogloss'))
cm_xgb_df = check_false_pred(xgb_y_pred)
cm_xgb_df

In [ ]:
rf_y_pred = model_fit_kfold(RandomForestClassifier())
cm_rf_df = check_false_pred(rf_y_pred)
cm_rf_df

In [ ]:
np.unique(y)

In [ ]:
play_song_ls(cm_rf_df)

## 이미지로 보는 분류 작업은 지속 예정

In [ ]:
img_path = "../imgs"
plt.figure(figsize=(14,10))
for idx, name in enumerate(cm_rf_df['name'].tolist()):
    plt.subplot(6,3,idx+1)
    plt.imshow(cv2.imread(f"{img_path}/melspec/{name}.png"))
    plt.title(name)
plt.subplots_adjust(wspace=0.2, hspace=0.9)
plt.show()